In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import numpy as np
import networkx as nx
import pylab as plt

eventsFrance = pd.read_json('/content/drive/My Drive/bachelor data/events/events_France.json')
matchesFrance = pd.read_json("/content/drive/My Drive/bachelor data/matches/matches_France.json")
tagsName = pd.read_csv("/content/drive/My Drive/bachelor data/tags2name.csv")
with open('/content/drive/My Drive/bachelor data/events/events_France.json') as f:
    d = json.load(f)
tags = json_normalize(data=d, record_path='tags', meta=['id'], record_prefix='_')
teams = pd.read_json("/content/drive/My Drive/bachelor data/teams.json")
eventsFranceTags = pd.merge(eventsFrance,
                       tags,
                       on='id')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]


In [ ]:
def checkZones2(dataframe, columnName):
  resultSeries = pd.Series([])
  i = -1
  for index, row in dataframe.iterrows():
    x = (row[columnName])['x']
    y = (row[columnName])['y']
    index = i + 1
    resultSeries = resultSeries.append(pd.Series([int(((x // 20) * 3) + 1 + (y // 33.3))]), ignore_index=True)
  return resultSeries

In [ ]:
# Filling 2D matrix of Barcelona team with corresponding passes
def get2dMatrix(passesDataframe, senderColumnName, RecipientColumnName):
  arr = np.arange(225).reshape(15,15)
  i = 0
  j = 0
  passCount = 0
  for i in range(15):
    for j in range(15):
      passCount = 0
      for index, row in passesDataframe.iterrows():
        if(row[senderColumnName] == i+1 and  row[RecipientColumnName] == j+1):
          passCount += 1
        arr[i][j] = passCount    
      j += 1
    i += 1 
  return arr

In [ ]:
def getTeamMatrix(teamId, teamMatchId):
  
  # Getting Successful passes of specific team in specific match
  successfulPasses = eventsFranceTags[(eventsFranceTags['teamId'] == teamId) & (eventsFranceTags['matchId'] == teamMatchId) & (eventsFranceTags['eventId'] == 8) & (eventsFranceTags['_id'] == 1801)]
  successfulPasses.reset_index(level=0, inplace=True)

  # Adding Sender/Recipient positions
  successfulPasses2 = successfulPasses['positions'].sum()
  del successfulPasses2[1::2]
  x2 = pd.Series(successfulPasses2)
  successfulPassesSender = pd.concat([successfulPasses, pd.DataFrame(x2)], axis=1)
  successfulPassesSender.rename(columns = {0:'Sender'}, inplace = True)

  successfulPasses1 = successfulPassesSender['positions'].sum()
  del successfulPasses1[0::2]
  x = pd.Series(successfulPasses1)
  successfulPassesSenderRecipient = pd.concat([successfulPassesSender, pd.DataFrame(x)], axis=1)
  successfulPassesSenderRecipient.rename(columns = {0:'Recepient'}, inplace = True) 

  # Adding Sender/Recipient zones

  zoneSeriesSender = checkZones2(successfulPassesSenderRecipient, 'Sender')
  zoneDataframeSender = pd.DataFrame(zoneSeriesSender)
  zoneDataframeSender.rename(columns = {0:'SenderZone'}, inplace = True)

  zoneSeriesRecipient = checkZones2(successfulPassesSenderRecipient, 'Recepient')
  zoneDataframeRecipient = pd.DataFrame(zoneSeriesRecipient)
  zoneDataframeRecipient.rename(columns = {0:'RecipientZone'}, inplace = True)

  successfulPassesSenderRecipientZ = pd.concat([successfulPassesSenderRecipient, zoneDataframeSender], axis=1)
  successfulPassesSenderRecipientZone = pd.concat([successfulPassesSenderRecipientZ, zoneDataframeRecipient], axis=1)
  
  # returning the 2D passes Matrix
  return get2dMatrix(successfulPassesSenderRecipientZone, 'SenderZone', 'RecipientZone').flatten()

In [ ]:
def getTeamMatches(teamId):
  teamMatchesIds = np.array([])
  eventsFranceTeam = eventsFrance[eventsFrance['teamId'] == teamId]
  teamMatchesIds = eventsFranceTeam.matchId.unique()
  return teamMatchesIds

In [ ]:
def getTeamMatrices(teamId):
  teamMatches = getTeamMatches(teamId)
  teamMatrices = np.zeros((len(teamMatches), 225))
  for i in range(len(teamMatches)):
    teamMatrices[i] = getTeamMatrix(teamId, teamMatches[i])
  return teamMatrices

In [ ]:
psgMatrices = np.zeros((38, 225))
psgMatrices = getTeamMatrices(3767)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
psgMatrices

array([[ 2.,  4.,  3., ...,  2., 10., 15.],
       [ 6.,  4.,  3., ...,  2.,  8.,  8.],
       [ 3.,  2.,  1., ...,  3., 11., 16.],
       ...,
       [ 7.,  5.,  3., ...,  4.,  6., 11.],
       [ 3.,  2.,  1., ...,  3.,  3.,  9.],
       [ 5.,  3.,  1., ...,  3.,  6.,  8.]])

In [ ]:
monacoMatrices = np.zeros((38, 225))
monacoMatrices = getTeamMatrices(19830)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
monacoMatrices

array([[ 3.,  1.,  1., ...,  7.,  9., 10.],
       [ 2.,  0.,  2., ...,  1.,  6., 11.],
       [ 0.,  0.,  0., ...,  1.,  8.,  9.],
       ...,
       [ 4.,  4.,  0., ...,  2.,  6., 17.],
       [ 3.,  0.,  1., ...,  3.,  4., 18.],
       [ 2.,  3.,  0., ...,  1., 10., 10.]])

In [ ]:
lyonMatrices = np.zeros((38, 225))
lyonMatrices = getTeamMatrices(3766)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
lyonMatrices

array([[ 0.,  3.,  0., ...,  2., 10., 21.],
       [ 3.,  7.,  1., ...,  3.,  3.,  9.],
       [ 9.,  4.,  2., ...,  1.,  4.,  3.],
       ...,
       [ 2.,  2.,  0., ...,  2.,  3., 11.],
       [ 2.,  7.,  0., ...,  0.,  3., 10.],
       [ 3.,  2.,  2., ...,  5.,  7., 20.]])

In [ ]:
marseilleMatrices = np.zeros((38, 225))
marseilleMatrices = getTeamMatrices(3771)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
marseilleMatrices

array([[ 6.,  6.,  3., ...,  3.,  5.,  2.],
       [ 3.,  8.,  3., ...,  2.,  6., 19.],
       [ 0.,  3.,  1., ...,  2.,  4.,  9.],
       ...,
       [ 1.,  3.,  1., ...,  1.,  6., 12.],
       [ 8.,  3.,  0., ...,  3.,  2., 10.],
       [ 0.,  6.,  0., ...,  2., 16., 21.]])

In [ ]:
rennesMatrices = np.zeros((38, 225))
rennesMatrices = getTeamMatrices(3774)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
rennesMatrices

array([[ 3.,  4.,  1., ...,  5.,  4.,  9.],
       [ 0.,  3.,  0., ...,  2.,  3.,  7.],
       [ 2.,  2.,  0., ...,  0.,  2.,  5.],
       ...,
       [ 4.,  3.,  1., ...,  2.,  3., 14.],
       [ 3.,  4.,  0., ...,  2.,  2., 14.],
       [ 7.,  3.,  2., ...,  0.,  4., 17.]])

In [ ]:
bordeauxMatrices = np.zeros((38, 225))
bordeauxMatrices = getTeamMatrices(3772)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
bordeauxMatrices

array([[ 4.,  3.,  1., ...,  2.,  4., 10.],
       [ 4.,  3.,  1., ...,  3.,  7., 22.],
       [ 1.,  1.,  1., ...,  1.,  4., 22.],
       ...,
       [ 3.,  6.,  3., ...,  5.,  2., 12.],
       [ 0.,  2.,  1., ...,  4.,  3.,  9.],
       [ 4.,  7.,  1., ...,  1.,  6.,  9.]])

In [ ]:
saintEtienneMatrices = np.zeros((38, 225))
saintEtienneMatrices = getTeamMatrices(3782)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
saintEtienneMatrices

array([[ 9.,  5.,  1., ...,  2.,  2.,  7.],
       [ 7.,  3.,  2., ...,  2.,  3.,  9.],
       [ 2.,  1.,  4., ...,  0.,  1.,  7.],
       ...,
       [ 3.,  2.,  3., ...,  0.,  1.,  5.],
       [ 4.,  3.,  3., ...,  2.,  3.,  9.],
       [ 3., 16.,  1., ...,  2.,  8., 26.]])

In [ ]:
niceMatrices = np.zeros((38, 225))
niceMatrices = getTeamMatrices(3775)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
niceMatrices

array([[ 6.,  5.,  2., ...,  3., 13.,  7.],
       [ 3.,  2.,  0., ...,  1.,  8., 21.],
       [ 7.,  1.,  3., ...,  1.,  4.,  4.],
       ...,
       [ 2.,  5.,  3., ...,  2.,  2.,  6.],
       [ 9.,  7.,  2., ...,  0.,  9., 11.],
       [ 6.,  3.,  1., ...,  0.,  7.,  2.]])

In [ ]:
nantesMatrices = np.zeros((38, 225))
nantesMatrices = getTeamMatrices(3770)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
nantesMatrices

array([[ 6.,  5.,  0., ...,  2.,  1.,  8.],
       [ 5.,  1.,  0., ...,  2.,  4.,  3.],
       [ 0.,  2.,  1., ...,  2.,  5., 11.],
       ...,
       [ 2.,  1.,  2., ...,  2.,  3.,  9.],
       [ 4.,  0.,  2., ...,  4.,  9., 18.],
       [ 3.,  1.,  1., ...,  1.,  4.,  6.]])

In [ ]:
montpellierMatrices = np.zeros((38, 225))
montpellierMatrices = getTeamMatrices(3787)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
montpellierMatrices

array([[ 0.,  6.,  1., ...,  3.,  4.,  6.],
       [ 2.,  3.,  1., ...,  0.,  1., 10.],
       [ 5.,  5.,  0., ...,  3.,  1.,  8.],
       ...,
       [ 5.,  4.,  2., ...,  1.,  2.,  3.],
       [ 4.,  8.,  4., ...,  3., 10.,  6.],
       [ 7.,  3.,  2., ...,  2.,  1.,  4.]])

In [ ]:
dijonMatrices = np.zeros((38, 225))
dijonMatrices = getTeamMatrices(3804)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
dijonMatrices

array([[ 0.,  0.,  1., ...,  0.,  6.,  3.],
       [ 2.,  5.,  2., ...,  6.,  4., 13.],
       [ 2.,  4.,  2., ...,  2.,  4.,  5.],
       ...,
       [ 2.,  8.,  1., ...,  1.,  3.,  4.],
       [ 0.,  5.,  2., ...,  1.,  3.,  6.],
       [ 5.,  3.,  5., ...,  0.,  1.,  1.]])

In [ ]:
guingumpMatrices = np.zeros((38, 225))
guingumpMatrices = getTeamMatrices(3785)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
guingumpMatrices

array([[1., 3., 0., ..., 0., 3., 1.],
       [0., 1., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 3., 7.],
       ...,
       [3., 1., 0., ..., 1., 0., 3.],
       [1., 2., 0., ..., 0., 2., 9.],
       [0., 0., 0., ..., 0., 2., 2.]])

In [ ]:
amiensMatrices = np.zeros((38, 225))
amiensMatrices = getTeamMatrices(3789)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
amiensMatrices

array([[ 4.,  4.,  1., ...,  1.,  3.,  3.],
       [ 5.,  4.,  2., ...,  0.,  5., 13.],
       [ 5.,  4.,  1., ...,  2.,  3.,  5.],
       ...,
       [ 3.,  4.,  1., ...,  2.,  5.,  7.],
       [ 1.,  4.,  2., ...,  0.,  5., 11.],
       [ 4.,  5.,  3., ...,  0.,  0.,  4.]])

In [ ]:
angersMatrices = np.zeros((38, 225))
angersMatrices = getTeamMatrices(3799)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
angersMatrices

array([[ 7.,  6.,  0., ...,  1.,  3.,  3.],
       [ 4.,  2.,  1., ...,  0.,  4.,  6.],
       [ 6.,  1.,  1., ...,  2.,  2.,  4.],
       ...,
       [ 3.,  3.,  2., ...,  0.,  4.,  9.],
       [ 5.,  3.,  0., ...,  3.,  7.,  5.],
       [ 4.,  7.,  3., ...,  2.,  2., 10.]])

In [ ]:
strasburgMatrices = np.zeros((38, 225))
strasburgMatrices = getTeamMatrices(3779)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
strasburgMatrices

array([[ 5.,  3.,  1., ...,  1.,  3.,  4.],
       [ 0.,  1.,  1., ...,  3.,  2.,  7.],
       [ 5.,  5.,  1., ...,  1.,  0.,  0.],
       ...,
       [ 4.,  7.,  1., ...,  1.,  2.,  3.],
       [ 2.,  2.,  0., ...,  2.,  3.,  1.],
       [11.,  2.,  3., ...,  0.,  3., 17.]])

In [ ]:
caenMatrices = np.zeros((38, 225))
caenMatrices = getTeamMatrices(3783)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
caenMatrices

array([[ 6.,  2.,  0., ...,  1.,  1.,  1.],
       [ 6.,  3.,  3., ...,  4.,  1., 10.],
       [ 8.,  1.,  0., ...,  0.,  3.,  1.],
       ...,
       [ 4.,  3.,  0., ...,  3.,  2.,  5.],
       [ 1.,  3.,  0., ...,  1.,  3.,  7.],
       [ 5.,  2.,  0., ...,  1.,  5.,  4.]])

In [ ]:
lilleMatrices = np.zeros((38, 225))
lilleMatrices = getTeamMatrices(3776)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
lilleMatrices

array([[ 8.,  4.,  5., ...,  1.,  4.,  8.],
       [ 7.,  5.,  0., ...,  0.,  5.,  9.],
       [ 3.,  1.,  2., ...,  4.,  5.,  8.],
       ...,
       [ 1.,  3.,  0., ...,  1.,  1.,  9.],
       [ 3.,  3.,  1., ...,  2., 11., 19.],
       [ 2.,  1.,  4., ...,  1., 17., 25.]])

In [ ]:
touloseMatrices = np.zeros((38, 225))
touloseMatrices = getTeamMatrices(3780)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
touloseMatrices

array([[ 1.,  2.,  0., ...,  0.,  1.,  6.],
       [ 2.,  3.,  0., ...,  3.,  1., 10.],
       [ 0.,  0.,  1., ...,  1.,  2.,  3.],
       ...,
       [ 5.,  1.,  1., ...,  0.,  2.,  2.],
       [ 4.,  2.,  1., ...,  1.,  1.,  4.],
       [ 3.,  1.,  1., ...,  2.,  3.,  4.]])

In [ ]:
troyesMatrices = np.zeros((38, 225))
troyesMatrices = getTeamMatrices(3795)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
troyesMatrices

array([[ 7.,  2.,  1., ...,  0.,  1.,  0.],
       [ 4.,  2.,  0., ...,  2.,  3.,  8.],
       [ 2.,  4.,  2., ...,  0.,  8.,  7.],
       ...,
       [ 8.,  3.,  1., ...,  2.,  6., 16.],
       [ 8.,  7.,  0., ...,  0.,  7., 12.],
       [ 4.,  5.,  1., ...,  6.,  6., 21.]])

In [ ]:
metzMatrices = np.zeros((38, 225))
metzMatrices = getTeamMatrices(3777)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
metzMatrices

array([[ 1.,  4.,  1., ...,  2.,  4., 14.],
       [ 0.,  1.,  0., ...,  2.,  4., 12.],
       [ 2.,  4.,  0., ...,  1.,  2., 14.],
       ...,
       [ 5.,  2.,  1., ...,  1.,  0.,  5.],
       [ 6.,  1.,  1., ...,  4.,  7., 13.],
       [ 5.,  2.,  1., ...,  0.,  4., 12.]])

In [ ]:
from google.colab import files
import pickle
with open('Ligue1.pickle', 'wb') as f:
  pickle.dump([psgMatrices, monacoMatrices, lyonMatrices, marseilleMatrices, rennesMatrices, bordeauxMatrices, saintEtienneMatrices, niceMatrices, nantesMatrices, montpellierMatrices, dijonMatrices, guingumpMatrices, amiensMatrices, angersMatrices, strasburgMatrices, caenMatrices, lilleMatrices, touloseMatrices, troyesMatrices, metzMatrices], f)
files.download('Ligue1.pickle')  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>